<a href="https://colab.research.google.com/github/LorenzoBoccalon/N-queens-problem/blob/master/N_queens_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [199]:
"""https://arxiv.org/pdf/1802.02006.pdf"""
import numpy as np
import itertools
import warnings
warnings.filterwarnings("ignore")
import time
from tqdm import tqdm
from typing import List, Tuple, Optional

# create global random number generator
RNG = np.random.default_rng(seed=42)

# type aliasing
Individual = List[int]
Pair = Tuple[Individual, Individual]
Point2D = Tuple[int, int]
Table = List[List[int]]


def genome_to_table(individual: Individual) -> Table:
    """transform genome to binary 2D-array"""
    nq = len(individual)
    tbl = np.zeros((nq, nq), np.int8)
    for r, c in enumerate(individual):
        tbl[r, c] = 1
    return tbl


def diag_attack(x: Point2D, y: Point2D) -> bool:
    """formula to check if coordinates intersect along a diagonal"""
    return abs(x[0] - y[0]) == abs(x[1] - y[1])


def count_attacks(genome: Individual) -> int:
    """given a chessboard disposition return the number of attacks.

    min = 0, no attacks
    max = NQ choose 2, i.e. every queen attack all the others"""
    cs = genome_to_coordinates(genome)
    tot = 0
    # generator of all pair combinations
    all_pairs = itertools.combinations(cs, 2)
    for c1, c2 in all_pairs:
        if c1[0] == c2[0]:  # same row
            tot += 1
        elif c1[1] == c2[1]:  # same col
            tot += 1
        elif diag_attack(c1, c2):  # same diag
            tot += 1
    return tot


def generate_genome(NQ: int = 8) -> Individual:
    """generate a random chessboard with NQ queens"""
    return RNG.permutation(NQ)


def genome_to_coordinates(genome: Individual) -> List[Point2D]:
    """given a genome return list of coordinates of the queens"""
    return np.array([(r, c) for r, c in enumerate(genome)])


def genetic_algo_solution(NQ: int = 8, NP: int = 100):
    """given NQ queens find a chessboard disposition where each queen does not attack any other. Use a genetic algorithm approach with a population of NP individuals per generation"""
    # set a limit
    max_iterations = 1_000_000
    solution = None
    pop = init_population(NQ, NP)
    for t in tqdm(range(max_iterations)):
        new_pop = []
        fs = fitness(pop)
        solution = check_population(pop, fs)
        if solution is not None:
            return solution
        ps = fitness_to_prob(fs)
        for _ in range(NP):
            (p1, p2) = selection(pop, ps)
            children = crossover((p1, p2)), crossover((p2, p1))
            children = [mutation(c) for c in children]
            new_pop = new_pop + children
        pop = new_pop


def init_population(NQ: int = 8, NP: int = 100) -> List[Individual]:
    """generate NP genomes of length NQ"""
    return [generate_genome(NQ) for _ in range(NP)]


def check_population(
    individuals: List[Individual], fitness: List[int]
) -> Optional[Individual]:
    """check if solution was found and eventualy return it"""
    for i, f in zip(individuals, fitness):
        if f == 0:
            return i
    return


def fitness(individuals: List[Individual]) -> List[int]:
    """given individuals return their fitness values"""
    # we use `count_attacks`, which is best if returns 0
    # therefore we minimise the fitness
    return [count_attacks(i) for i in individuals]


def fitness_to_prob(fitness: List[int]) -> List[float]:
    """transform list of fitness values into probabilities"""
    # numpy wrapping
    fitness = np.array(fitness)
    # add small quantity to prevent 0 div
    fitness = fitness + 1e-10
    # inverse function
    probs = 1 / fitness
    # normalise into probabilities'
    probs = probs / probs.sum()
    return probs.tolist()


def selection(individuals: List[Individual], probs: List[float]) -> Pair:
    """given individuals and probabilities of being chosen return a pair for reproduction"""
    p1, p2 = RNG.choice(individuals, size=2, replace=False, p=probs)
    return p1, p2


def crossover(parents: Pair, proportion: float = 0.5, debug: bool = False) -> Individual:
    """given parents pair generate a child"""
    p1, p2 = parents
    nq = len(p1)
    chunk_size = int(nq * proportion)
    split_point = RNG.choice(range(0, nq - chunk_size))
    start, end = split_point, split_point + chunk_size
    p1_genes = np.array(p1[start:end], np.int8)
    p2_genes = np.array([g for g in p2 if g not in p1_genes], np.int8)
    if(debug):
        print(f"split_point={split_point}, chunk_size={chunk_size}")
        print(f"p1_genes={p1_genes}, p2_genes={p2_genes}")
    child = np.concatenate((
        p2_genes[:start],
        p1_genes,
        p2_genes[start:],
    ))
    return child


def mutation_alternative(individual: Individual, prob: float = 1e-3) -> Individual:
    """mutate individual with small probability"""
    # toss unbalanced coin
    mutate_prob = prob
    dont_mutate_prob = 1 - prob
    if RNG.choice(2, p=[dont_mutate_prob, mutate_prob]):
        nq = len(individual)
        # select a gene (uniform prob)
        pos = RNG.choice(nq)
        # generate new gene
        old_gene = individual[pos]
        # sum and modulo trick to prevent the same gene
        new_gene = (old_gene + RNG.choice(range(1, nq))) % nq
        individual[pos] = new_gene
    return individual

def mutation(individual: Individual, prob: float = 1e-2) -> Individual:
    """mutate individual with small probability"""
    # toss unbalanced coin
    mutate_prob = prob
    dont_mutate_prob = 1 - prob
    if RNG.choice(2, p=[dont_mutate_prob, mutate_prob]):
        # swap two genes
        nq = len(individual)
        a, b = RNG.choice(nq, 2, replace=False)
        individual[a], individual[b] = individual[b], individual[a]
    return individual


print("Genetic Algorithm")
start = time.perf_counter()
solution = genetic_algo_solution(NQ=16)
table = genome_to_table(solution)
delta = time.perf_counter() - start
print()
print(f"Solution found in {delta} seconds!")
print(table)
print(f"Genome: {solution}")

Genetic Algorithm


  0%|          | 588/1000000 [00:49<23:23:17, 11.87it/s]


Solution found in 49.546697220997885 seconds!
[[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]]
Genome: [ 6 12  1 15  5  3  9 14  2 11  7  0  4 13 10  8]
